In [51]:
%pip install matplotlib
%pip install shapefile
%pip install requests
%pip install Basemap
%pip install numpy

clear_output()

In [70]:
from mpl_toolkits.basemap import Basemap
from IPython.display import clear_output
from io import BytesIO, StringIO
import matplotlib.patches as pts
import matplotlib.pyplot as plt
import shapefile as spf
import tifffile as tiff
import numpy as np
import requests

In [44]:
url_VDL = 'https://github.com/LanceryH/NASA-hackathon-2024/raw/main/data_VDL.tif'

response = requests.get(url_VDL)
response.raise_for_status()

tiff_data = BytesIO(response.content)
tif_image = tiff.imread(tiff_data)

In [72]:
url_GMMRD = 'https://github.com/LanceryH/NASA-hackathon-2024/raw/main/data_GMMRD.asc'

# Fetch the .asc file
response = requests.get(url_GMMRD)
response.raise_for_status()  # Ensure the request was successful

# Use StringIO for text-based .asc files
asc_data = StringIO(response.text)

# Function to read and parse the .asc file
def read_asc(file_like):
    # Skip header lines
    header_lines = 6
    for _ in range(header_lines):
        file_like.readline()
        
    # Read the grid data
    data = np.loadtxt(file_like)
    return data

# Read the .asc data
data = read_asc(asc_data)

ValueError: could not convert string '<!DOCTYPE' to float64 at row 0, column 1.

In [33]:
# Define chunk size and starting points
chunk_size = 5000
start_row, start_col = 9000, 13000
end_row, end_col = 19000, 28000 

lat_min, lat_max = -90, 90   # Latitude range for the whole image
lon_min, lon_max = -180, 180 # Longitude range for the whole image

# Define the size of the full image
image_height, image_width = tif_image.shape[:2]

# Calculate the geographic extent for your chunk
lon_min = (start_col * 360 / image_width) - 180
lon_max = (end_col * 360 / image_width) - 180
lat_min = -(end_row * 180 / image_height) + 90
lat_max = -(start_row * 180 / image_height) + 90

In [34]:
# Function to read a chunk of the image
def read_chunk(tif_image, start_row, start_col, end_row, end_col):
    return tif_image[start_row:end_row, start_col:end_col]

In [35]:
chunk_image = read_chunk(tif_image, start_row, start_col, end_row, end_col)

In [36]:
plt.figure()
m = Basemap(projection='cyl',resolution="h",
                llcrnrlat=lat_min, urcrnrlat=lat_max,
                llcrnrlon=lon_min, urcrnrlon=lon_max)
m.drawcountries(color="yellow")
m.drawstates(color="yellow")
# m.drawcoastlines(color="yellow")
plt.imshow(chunk_image, extent=[lon_min, lon_max, lat_min, lat_max], interpolation='none')
plt.tight_layout()
plt.show()

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
